In [1]:
import tensorflow as tf
import pandas as pd
from PIL import Image
import numpy as np
import zipfile
import os
import glob
import random
import sys

import skimage.io  #Used for imshow function
import skimage.transform  #Used for resize function
from skimage.morphology import label  #Used for Run-Length-Encoding RLE to create final submission
import matplotlib.pyplot as plt

from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

2023-12-09 20:11:40.123091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 20:11:40.123128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 20:11:40.124602: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 20:11:40.132402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 20:11:41.146950: W tensorflow/compiler/tf2

In [65]:
# Загрузка данных
data_dir = 'data/data_256/'
images_dir = 'images/'
masks_dir = 'masks/'
WIDTH = 256
HEIGHT = 256
CLASSES = 2

In [66]:
# def resize_rectangle(rect, orig_size = (2444, 1292), new_size=(WIDTH, HEIGHT)):
#     orig_width = orig_size[0]
#     orig_height = orig_size[1]
#     new_width = new_size[0]
#     new_height = new_size[1]w
# 
#     xmin, ymin, xmax, ymax = rect
# 
#     # Вычисляем масштабные коэффициенты для изменения размера
#     width_scale = new_width / 1. /orig_width
#     height_scale = new_height / 1. /orig_height
# 
#     # Пересчитываем координаты прямоугольника
#     new_xmin = int(xmin * width_scale)
#     new_ymin = int(ymin * height_scale)
#     new_xmax = int(xmax * width_scale)
#     new_ymax = int(ymax * height_scale)
# 
#     return new_xmin, new_ymin, new_xmax, new_ymax


In [67]:
import os
from PIL import Image
import numpy as np


def load_image(image_path):
    image = Image.open(data_dir + images_dir + image_path)  #.convert('L')  # Преобразуем в оттенки серого
    return np.array(image)


def load_mask(mask_path):
    mask = Image.open(data_dir + masks_dir + mask_path).convert('L')  # Преобразуем в оттенки серого
    mask_array = np.array(mask)
    mask_float = mask_array.astype(float) / 255.0  # Преобразуем в тип float и нормализуем значения до диапазона [0, 1]
    return mask_float


In [68]:
# # Подготовка данных
image_paths = sorted(os.listdir(data_dir + images_dir))

In [69]:
print(image_paths)

['1.png', '10.png', '100.png', '101.png', '102.png', '103.png', '104.png', '105.png', '106.png', '107.png', '108.png', '109.png', '11.png', '110.png', '111.png', '112.png', '113.png', '114.png', '115.png', '116.png', '117.png', '118.png', '119.png', '12.png', '120.png', '121.png', '122.png', '123.png', '124.png', '125.png', '126.png', '127.png', '128.png', '129.png', '13.png', '130.png', '131.png', '132.png', '133.png', '134.png', '135.png', '136.png', '137.png', '138.png', '139.png', '14.png', '140.png', '141.png', '142.png', '143.png', '144.png', '145.png', '146.png', '147.png', '148.png', '149.png', '15.png', '150.png', '151.png', '152.png', '153.png', '154.png', '155.png', '156.png', '157.png', '158.png', '159.png', '16.png', '160.png', '161.png', '162.png', '163.png', '164.png', '165.png', '166.png', '167.png', '168.png', '169.png', '17.png', '170.png', '171.png', '172.png', '173.png', '174.png', '175.png', '176.png', '177.png', '178.png', '179.png', '18.png', '180.png', '181.png'

In [70]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую, валидационную и тестовую выборки
train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)
train_paths, valid_paths = train_test_split(train_paths, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Проверка размеров выборок
print("Train samples:", len(train_paths))
print("Valid samples:", len(valid_paths))
print("Test samples:", len(test_paths))

Train samples: 358
Valid samples: 120
Test samples: 120


In [71]:
train_images = [load_image(image_path) for image_path in train_paths]
train_masks = [load_mask(mask_path) for mask_path in train_paths]
train_images = np.array(train_images)
train_masks = np.array(train_masks)

In [72]:
valid_images = [load_image(image_path) for image_path in valid_paths]
valid_masks = [load_mask(mask_path) for mask_path in valid_paths]
valid_images = np.array(valid_images)
valid_masks = np.array(valid_masks)

In [73]:
test_images = [load_image(image_path) for image_path in test_paths]
test_masks = [load_mask(mask_path) for mask_path in test_paths]
test_images = np.array(test_images)
test_masks = np.array(test_masks)

In [74]:
print(test_masks[0][100])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.49019608 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.94117647 0.90588235 1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         

In [79]:

def unet_test_model(classes: int = 1):
    tf.keras.backend.clear_session()
    nb_filter = [32, 64, 128, 256, 512]
    # Build U-Net++ model
    inputs = Input((WIDTH, HEIGHT, 3))
    s = Lambda(lambda x: x / 255)(inputs)

    c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.5)(c1)
    c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c1)
    c1 = Dropout(0.5)(c1)
    p1 = MaxPooling2D((2, 2), strides=(2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.5)(c2)
    c2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c2)
    c2 = Dropout(0.5)(c2)
    p2 = MaxPooling2D((2, 2), strides=(2, 2))(c2)

    up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding='same')(c2)
    conv1_2 = concatenate([up1_2, c1], name='merge12', axis=3)
    c3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_2)
    c3 = Dropout(0.5)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c3)
    c3 = Dropout(0.5)(c3)

    conv3_1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p2)
    conv3_1 = Dropout(0.5)(conv3_1)
    conv3_1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_1)
    conv3_1 = Dropout(0.5)(conv3_1)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

    up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
    conv2_2 = concatenate([up2_2, c2], name='merge22', axis=3)  #x10
    conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_2)
    conv2_2 = Dropout(0.5)(conv2_2)
    conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_2)
    conv2_2 = Dropout(0.5)(conv2_2)

    up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
    conv1_3 = concatenate([up1_3, c1, c3], name='merge13', axis=3)
    conv1_3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_3)
    conv1_3 = Dropout(0.5)(conv1_3)
    conv1_3 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_3)
    conv1_3 = Dropout(0.5)(conv1_3)

    conv4_1 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4_1 = Dropout(0.5)(conv4_1)
    conv4_1 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv4_1)
    conv4_1 = Dropout(0.5)(conv4_1)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

    up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
    conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=3)  #x20
    conv3_2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_2)
    conv3_2 = Dropout(0.5)(conv3_2)
    conv3_2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_2)
    conv3_2 = Dropout(0.5)(conv3_2)

    up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
    conv2_3 = concatenate([up2_3, c2, conv2_2], name='merge23', axis=3)
    conv2_3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_3)
    conv2_3 = Dropout(0.5)(conv2_3)
    conv2_3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_3)
    conv2_3 = Dropout(0.5)(conv2_3)

    up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
    conv1_4 = concatenate([up1_4, c1, c3, conv1_3], name='merge14', axis=3)
    conv1_4 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_4)
    conv1_4 = Dropout(0.5)(conv1_4)
    conv1_4 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_4)
    conv1_4 = Dropout(0.5)(conv1_4)

    conv5_1 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5_1 = Dropout(0.5)(conv5_1)
    conv5_1 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv5_1)
    conv5_1 = Dropout(0.5)(conv5_1)

    up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
    conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=3)  #x30
    conv4_2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv4_2)
    conv4_2 = Dropout(0.5)(conv4_2)
    conv4_2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv4_2)
    conv4_2 = Dropout(0.5)(conv4_2)

    up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
    conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=3)
    conv3_3 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_3)
    conv3_3 = Dropout(0.5)(conv3_3)
    conv3_3 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv3_3)
    conv3_3 = Dropout(0.5)(conv3_3)

    up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
    conv2_4 = concatenate([up2_4, c2, conv2_2, conv2_3], name='merge24', axis=3)
    conv2_4 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_4)
    conv2_4 = Dropout(0.5)(conv2_4)
    conv2_4 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv2_4)
    conv2_4 = Dropout(0.5)(conv2_4)

    up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
    conv1_5 = concatenate([up1_5, c1, c3, conv1_3, conv1_4], name='merge15', axis=3)
    conv1_5 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_5)
    conv1_5 = Dropout(0.5)(conv1_5)
    conv1_5 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(conv1_5)
    conv1_5 = Dropout(0.5)(conv1_5)

    nestnet_output_4 = Conv2D(classes, (1, 1), activation='sigmoid', kernel_initializer='he_normal', name='output_4',
                              padding='same')(conv1_5)
    # output = Conv2D(
    #     filters=classes,
    #     kernel_size=(3, 3),
    #     padding='same',
    #     activation='softmax',
    #     use_bias=True,
    #     kernel_initializer='glorot_uniform',
    #     name='final_conv',
    # )(nestnet_output_4)

    # create keras model instance
    return Model(inputs=[inputs], outputs=
    [nestnet_output_4])


In [80]:
LR = 0.0001


def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def bce_dice_loss(y_true, y_pred):
    return 0.5 * tf.keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

In [81]:
from keras.losses import binary_crossentropy
import keras.backend as K


def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


def weighted_dice_coeff(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    return score


def weighted_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = 1 - weighted_dice_coeff(y_true, y_pred, weight)
    return loss


def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))

    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) * \
           (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)


def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + (1 - weighted_dice_coeff(y_true, y_pred, weight))
    return loss

In [83]:
import tensorflow as tf
from tensorflow.keras.metrics import IoU

model = unet_test_model(CLASSES)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss=bce_dice_loss,
              metrics=[IoU(num_classes=CLASSES, target_class_ids=
              [i for i in range(CLASSES)])])

In [84]:
print(train_images)

[[[[249 249 249]
   [247 247 247]
   [247 247 247]
   ...
   [249 249 249]
   [248 248 248]
   [249 249 249]]

  [[245 245 245]
   [248 248 248]
   [255 255 255]
   ...
   [247 247 247]
   [248 248 248]
   [250 250 250]]

  [[249 249 249]
   [229 229 229]
   [126 126 126]
   ...
   [246 246 246]
   [246 246 246]
   [250 250 250]]

  ...

  [[ 70  70  70]
   [ 28  28  28]
   [ 21  21  21]
   ...
   [  5   5   5]
   [ 17  17  17]
   [ 46  46  46]]

  [[ 57  57  57]
   [ 22  22  22]
   [ 13  13  13]
   ...
   [  2   2   2]
   [ 12  12  12]
   [ 39  39  39]]

  [[ 88  88  88]
   [ 53  53  53]
   [ 43  43  43]
   ...
   [ 23  23  23]
   [ 38  38  38]
   [ 70  70  70]]]


 [[[196 196 196]
   [199 199 199]
   [198 198 198]
   ...
   [200 200 200]
   [200 200 200]
   [195 195 195]]

  [[250 250 250]
   [252 252 252]
   [255 255 255]
   ...
   [253 253 253]
   [253 253 253]
   [253 253 253]]

  [[246 246 246]
   [252 252 252]
   [206 206 206]
   ...
   [247 247 247]
   [249 249 249]
   [248 248

In [85]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=1, batch_size=4)


ValueError: in user code:

    File "/home/daniil/Important_catalog/SHAD/Python/Hell_Dragon/shad_env/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_62172/26868942.py", line 20, in bce_dice_loss  *
        loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    File "/home/daniil/Important_catalog/SHAD/Python/Hell_Dragon/shad_env/lib/python3.11/site-packages/keras/src/losses.py", line 2532, in binary_crossentropy
        backend.binary_crossentropy(y_true, y_pred, from_logits=from_logits),
    File "/home/daniil/Important_catalog/SHAD/Python/Hell_Dragon/shad_env/lib/python3.11/site-packages/keras/src/backend.py", line 5822, in binary_crossentropy
        return tf.nn.sigmoid_cross_entropy_with_logits(

    ValueError: `logits` and `labels` must have the same shape, received ((None, 256, 256, 2) vs (None, 256, 256)).


In [40]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=5, batch_size=4)


Epoch 1/5
90/90 [==============================] - 460s 5s/step - loss: 0.1126 - io_u: 0.4553 - val_loss: -0.0551 - val_io_u: 0.4586
Epoch 2/5
90/90 [==============================] - 459s 5s/step - loss: 0.0429 - io_u: 0.4553 - val_loss: -0.0628 - val_io_u: 0.4586
Epoch 3/5
90/90 [==============================] - 459s 5s/step - loss: 0.0141 - io_u: 0.4553 - val_loss: -0.0605 - val_io_u: 0.4586
Epoch 4/5
90/90 [==============================] - 459s 5s/step - loss: -0.0026 - io_u: 0.4553 - val_loss: -0.0669 - val_io_u: 0.4586
Epoch 5/5
90/90 [==============================] - 460s 5s/step - loss: -0.0154 - io_u: 0.4553 - val_loss: -0.0580 - val_io_u: 0.4586


In [26]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=20, batch_size=8)

Epoch 1/20
18/18 [==============================] - 86s 5s/step - loss: 0.0519 - mean_io_u: 0.4319 - val_loss: -0.0203 - val_mean_io_u: 0.4365
Epoch 2/20
18/18 [==============================] - 86s 5s/step - loss: 0.0308 - mean_io_u: 0.4319 - val_loss: 0.0596 - val_mean_io_u: 0.4365
Epoch 3/20
18/18 [==============================] - 86s 5s/step - loss: 0.0140 - mean_io_u: 0.4319 - val_loss: 0.0227 - val_mean_io_u: 0.4365
Epoch 4/20
18/18 [==============================] - 86s 5s/step - loss: 0.0083 - mean_io_u: 0.4319 - val_loss: 0.0867 - val_mean_io_u: 0.4365
Epoch 5/20
18/18 [==============================] - 86s 5s/step - loss: -0.0017 - mean_io_u: 0.4319 - val_loss: 0.0889 - val_mean_io_u: 0.4365
Epoch 6/20
18/18 [==============================] - 86s 5s/step - loss: -0.0157 - mean_io_u: 0.4319 - val_loss: 0.0578 - val_mean_io_u: 0.4365
Epoch 7/20
18/18 [==============================] - 86s 5s/step - loss: -0.0274 - mean_io_u: 0.4319 - val_loss: 0.1524 - val_mean_io_u: 0.4365
Ep

In [ ]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=20, batch_size=4)

In [ ]:
# Создание и обучение модели U-Net
model.fit(train_images, train_masks, validation_data=(valid_images, valid_masks), epochs=20, batch_size=4)

In [48]:
tf.saved_model.save(model, 'model')

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [23]:
result = model.predict(test_images)

2023-12-09 20:20:43.325928: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 805306368 exceeds 10% of free system memory.
2023-12-09 20:20:45.168510: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1073741824 exceeds 10% of free system memory.
2023-12-09 20:20:47.122942: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1342177280 exceeds 10% of free system memory.


1/4 [======>.......................] - ETA: 23s

2023-12-09 20:20:50.870183: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 805306368 exceeds 10% of free system memory.
2023-12-09 20:20:52.929968: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1073741824 exceeds 10% of free system memory.


4/4 [==============================] - 28s 7s/step


In [27]:
print(result[0])

[[[0.42203665]
  [0.35392913]
  [0.3018944 ]
  ...
  [0.34531984]
  [0.34214437]
  [0.25687236]]

 [[0.43551797]
  [0.41775978]
  [0.36844695]
  ...
  [0.44380885]
  [0.39099336]
  [0.19417508]]

 [[0.47164166]
  [0.43103456]
  [0.369371  ]
  ...
  [0.412446  ]
  [0.36045402]
  [0.18623644]]

 ...

 [[0.45005953]
  [0.17160515]
  [0.19931158]
  ...
  [0.14693075]
  [0.1785332 ]
  [0.24441239]]

 [[0.40026015]
  [0.22380106]
  [0.18568495]
  ...
  [0.20004755]
  [0.23183423]
  [0.24185789]]

 [[0.5070852 ]
  [0.3321939 ]
  [0.34704894]
  ...
  [0.28137147]
  [0.32106948]
  [0.34892485]]]


In [28]:
from PIL import Image
import numpy as np

# Предположим, что результат предсказания модели содержит вероятности пикселей для каждого класса
# Размерность результатов предсказания: (batch_size, HEIGHT, WIDTH, num_classes)

# Предположим, что результат предсказания модели содержит вероятности пикселей для каждого класса
# Размерность результатов предсказания: (batch_size, HEIGHT, WIDTH, num_classes)
num_classes = 7  # Количество классов (6 классов + 1 класс "ничего")
threshold = 0.3  # Порог для принятия решения о наличии объекта на пикселе

# Преобразование результатов предсказания в маски с максимальными значениями классов
predicted_masks = result
# true_masks = test_masks
# Создание палитры цветов для каждого класса
palette = [
    (0, 0, 0),  # Класс "ничего" (черный цвет)
    (255, 0, 0),  # Класс 1 (красный цвет)
    (0, 255, 0),  # Класс 2 (зеленый цвет)
    (0, 0, 255),  # Класс 3 (синий цвет)
    (255, 255, 0),  # Класс 4 (желтый цвет)
    (255, 0, 255),  # Класс 5 (пурпурный цвет)
    (0, 255, 255),  # Класс 6 (бирюзовый цвет)
    (255, 255, 255)
]
# 
# Создание изображений для каждого предсказанного класса
for i in range(min(predicted_masks.shape[0], 100)):
    image = Image.new("RGB",
                      (
                          WIDTH, HEIGHT
                      )
                      )
    # true_mask = Image.new("RGB",
    #                   (
    #                       WIDTH, HEIGHT
    #                   )
    #                   )
    for y in range(
            HEIGHT
    ):
        for x in range(
                WIDTH
        ):
            if predicted_masks[i][y][x] >= threshold:
                class_index = 7
            else:
                class_index = 0
            # val = predicted_masks[i][x][y]
            image.putpixel((x, y), palette[class_index])  #(int(val * 255), int(val * 255), int(val * 255)))
            # if true_masks[i][y][x] >= threshold:
            #     true_mask.putpixel((x, y), palette[7])
            # else:
            #     true_mask.putpixel((x, y), palette[0])
    image.save(f"result_small_5/predicted_image_{i}.png")
    # true_mask.save(f"result_small_5/predicted_image_{i}_true.png")



In [ ]:
print(test_masks[0])